In [ ]:
#Import required libraries
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup #don't forget to pip install BS

def cleantags(x):
    '''
    The purpose of this function is to clean the HTML tags off the text scraped from the website
    '''
    s1 = re.sub("<h2>", "", x)
    s2 = re.sub("</h2>", "", s1)
    s3 = re.sub("<strong>", "", s2)
    s4 = re.sub("</strong>", "", s3)
    return s4
def cleantagsdigits(x):
    '''
    The purpose of this function is to clean the HTML tags and digits that preceed the 
    YouTube channel name off the text scraped from the website
    '''
    s1 = re.sub("<h2>\d*\.\s", "", x)
    s2 = re.sub("</h2>", "", s1)
    s3 = re.sub("<strong>", "", s2)
    s4 = re.sub("</strong>", "", s3)
    return s4

In [ ]:
# Load thd page and get it into so yummy Beautiful Soup
html_doc = 'https://marketing4ecommerce.net/youtubers-mas-seguidos-de-espana-2019-50/'
page = requests.get("https://marketing4ecommerce.net/youtubers-mas-seguidos-de-espana-2019-50/")
soup = BeautifulSoup(html_doc, 'html.parser')
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('h2')

In [ ]:
#Get the data into a dataframe to start working some magic
df = pd.DataFrame(soup.find_all('h2'))
df = df.iloc[5:] #removed the first 5 rows as garbage
df.columns = ['raw_name'] #rename the column
df = df.drop([39,53], axis=0) #drop some more garbage rows


In [ ]:
df

In [ ]:
df['raw2'] = df['raw_name'] 
df.dtypes
df['raw_name'] = df['raw_name'].astype('str')  #convert dtype to string
df['raw2'] = df['raw2'].astype('str')  #convert dtype to string
df_bup = df #create backup
df["interim_name"] = df["raw_name"].apply(cleantags)
df['rank'] = df['interim_name'].apply(lambda s:s.split('.')[0])
df["interim_nameb"] = df["raw2"].apply(cleantagsdigits)
df['rankb'] = df['interim_nameb'].apply(lambda s:s.split('.')[0])

In [ ]:
df

In [ ]:
df_clean = df[['rank', 'rankb']]
df_clean = df_clean.rename({'rankb': 'Channel'}, axis=1) 
df_clean


In [ ]:
df_clean_bup = df_clean

df_clean.loc[7,'Channel'] = 'WillyRex'
df_clean.loc[10,'Channel']  = 'iTownGamePlay'
df_clean.loc[18,'Channel']  = 'DaniRep'
df_clean.loc[34,'Channel']  = 'MangelRogel'
df_clean.loc[35,'Channel']  = 'ElChurches'
df_clean.loc[22,'Channel']  =  'Adexe Nau'
df_clean.loc[7,'rank']  = ' 3'

df_clean

In [ ]:
df_clean['rank'] = df_clean.apply(lambda x: x.str.strip())

df_clean

In [ ]:
#Connection to DB
from sqlalchemy import create_engine
driver = 'mysql+pymysql'
ip = 'barcelona-db.cyxhqbnhiohl.eu-west-3.rds.amazonaws.com'
username = 'admin'
password = 'PercyA2019!'
db = 'project3'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
# Engine & Query
engine = create_engine(connection_string)

In [ ]:
df_clean = df_clean.reset_index(drop=True)
df_clean

In [ ]:
from sqlalchemy import exc
num_rows = len(df_clean)
#Iterate one row at a time
for i in range(num_rows):
    try:
        #Try inserting the row
        df_clean.iloc[i:i+1].to_sql(name="content_creator",con = engine,if_exists = 'append',index=False)
    except exc.IntegrityError:
        #Ignore duplicates
        pass